In [51]:
import pandas as pd
import xgboost
from sklearn.preprocessing import OrdinalEncoder

In [52]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [55]:
lang_feats = pd.read_excel("../../xtreme_langs_wals.xlsx", index_col=0)

In [57]:
lang_feats.head()

,ISO 693-2,wals_code,iso_code,glottocode,Name,latitude,longitude,genus,family,#feats,macroarea,countrycodes,20A Fusion of Selected Inflectional Formatives,21A Exponence of Selected Inflectional Formatives,22A Inflectional Synthesis of the Verb,23A Locus of Marking in the Clause,24A Locus of Marking in Possessive Noun Phrases,25A Locus of Marking: Whole-language Typology,26A Prefixing vs. Suffixing in Inflectional Morphology,27A Reduplication,28A Case Syncretism,29A Syncretism in Verbal Person/Number Marking,30A Number of Genders,31A Sex-based and Non-sex-based Gender Systems,32A Systems of Gender Assignment,33A Coding of Nominal Plurality,34A Occurrence of Nominal Plurality,35A Plurality in Independent Personal Pronouns,36A The Associative Plural,37A Definite Articles,38A Indefinite Articles,39A Inclusive/Exclusive Distinction in Independent Pronouns,40A Inclusive/Exclusive Distinction in Verbal Inflection,41A Distance Contrasts in Demonstratives,42A Pronominal and Adnominal Demonstratives,43A Third Person Pronouns and Demonstratives,44A Gender Distinctions in Independent Personal Pronouns,45A Politeness Distinctions in Pronouns,46A Indefinite Pronouns,47A Intensifiers and Reflexive Pronouns,48A Person Marking on Adpositions,49A Number of Cases,50A Asymmetrical Case-Marking,51A Position of Case Affixes,52A Comitatives and Instrumentals,53A Ordinal Numerals,54A Distributive Numerals,55A Numeral Classifiers,56A Conjunctions and Universal Quantifiers,57A Position of Pronominal Possessive Affixes,58A Obligatory Possessive Inflection,59A Possessive Classification,"60A Genitives, Adjectives and Relative Clauses",61A Adjectives without Nouns,62A Action Nominal Constructions,63A Noun Phrase Conjunction,64A Nominal and Verbal Conjunction,65A Perfective/Imperfective Aspect,66A The Past Tense,67A The Future Tense,68A The Perfect,69A Position of Tense-Aspect Affixes,70A The Morphological Imperative,71A The Prohibitive,72A Imperative-Hortative Systems,73A The Optative,74A Situational Possibility,75A Epistemic Possibility,76A Overlap between Situational and Epistemic Modal Marking,77A Semantic Distinctions of Evidentiality,78A Coding of Evidentiality,79A Suppletion According to Tense and Aspect,80A Verbal Number and Suppletion,"81A Order of Subject, Object and Verb",82A Order of Subject and Verb,83A Order of Object and Verb,"84A Order of Object, Oblique, and Verb",85A Order of Adposition and Noun Phrase,86A Order of Genitive and Noun,87A Order of Adjective and Noun,88A Order of Demonstrative and Noun,89A Order of Numeral and Noun,90A Order of Relative Clause and Noun,91A Order of Degree Word and Adjective,92A Position of Polar Question Particles,93A Position of Interrogative Phrases in Content Questions,94A Order of Adverbial Subordinator and Clause,95A Relationship between the Order of Object and Verb and the Order of Adposition and Noun Phrase,96A Relationship between the Order of Object and Verb and the Order of Relative Clause and Noun,97A Relationship between the Order of Object and Verb and the Order of Adjective and Noun,98A Alignment of Case Marking of Full Noun Phrases,99A Alignment of Case Marking of Pronouns,100A Alignment of Verbal Person Marking,101A Expression of Pronominal Subjects,102A Verbal Person Marking,103A Third Person Zero of Verbal Person Marking,104A Order of Person Markers on the Verb,105A Ditransitive Constructions: The Verb 'Give',106A Reciprocal Constructions,107A Passive Constructions,108A Antipassive Constructions,109A Applicative Constructions,110A Periphrastic Causative Constructions,111A Nonperiphrastic Causative Constructions,112A Negative Morphemes,113A Symmetric and Asymmetric Standard Negation,114A Subtypes of Asymmetric Standard Negation,115A Negative Indefinite Pronouns and Predicate Negation,116A Polar Questions,117A Predicative Possession,118A Predicative Adjectives,119A Nominal and Locational Predication,120A Zero Copula for Predicate Nominals,121A Comparative Constructions,122A Relativization on S

In [58]:
pair_scores = pd.read_csv("lang_pair_scores.csv")

In [59]:
pair_scores.head()

,train_lang,test_lang,acc,f1
0,af,af,0.984390,0.982330
1,af,ar,0.587932,0.517718
2,af,bg,0.847598,0.834094
3,af,de,0.837953,0.828865
4,af,el,0.800967,0.779682


In [74]:
feat_cols = lang_feats.columns[12:]

In [84]:
final_arr = []
final_cols = list(pair_scores.columns)+[col.split()[0]+"_train" for col in feat_cols]+ [col.split()[0]+"_test" for col in feat_cols]

for i in pair_scores.index:
    train_feats =  list(pair_scores.iloc[i,:].values) + list(lang_feats.loc[train_lang,feat_cols].values) + list(lang_feats.loc[test_lang,feat_cols].values)
    final_arr.append(train_feats)

final_df = pd.DataFrame(data=final_arr,columns=final_cols)
final_df.head()

Fill missing values

In [9]:
# TODO change df name in this line and below
lang_feats[feat_cols] = lang_feats[feat_cols].fillna("0 Missing")

Encode categorical variables (i.e. all of them)

In [10]:
mappings = []
for col in feat_cols:
    unique_vals = lang_feats[col].unique()
#     every value starts with an index number, additionally we mark missing as 0
    sorted_vals = sorted(unique_vals, key=lambda element: [element[0]])
    mappings.append(sorted_vals)
    print(sorted_vals)

['0 Missing', '1 Exclusively concatenative', '2 Exclusively isolating', '4 Tonal/isolating', '6 Ablaut/concatenative', '7 Isolating/concatenative']
['0 Missing', '1 Monoexponential case', '2 Case + number', '3 Case + referentiality', '5 No case']
['0 Missing', '1 0-1 category per word', '2 2-3 categories per word', '3 4-5 categories per word', '4 6-7 categories per word', '5 8-9 categories per word']
['0 Missing', '1 Head marking', '2 Dependent marking', '3 Double marking', '4 No marking']
['0 Missing', '1 Head marking', '2 Dependent marking', '3 Double marking', '4 No marking', '5 Other']
['0 Missing', '2 Dependent-marking', '3 Double-marking', '4 Zero-marking', '5 Inconsistent or other']
['0 Missing', '1 Little affixation', '2 Strongly suffixing', '3 Weakly suffixing', '4 Equal prefixing and suffixing', '5 Weakly prefixing']
['0 Missing', '1 Productive full and partial reduplication', '2 Full reduplication only', '3 No productive reduplication']
['0 Missing', '1 No case marking', '2 

In [14]:
oe = OrdinalEncoder(categories=mappings)
oe.fit(lang_feats[feat_cols])
lang_feats[feat_cols] = oe.transform(lang_feats[feat_cols])